# Breakwater blocks


## Problem
It is desired to design the cheapest breakwater block (box-shaped). The price of the brick will depend only on the amount of used material. It is required that each face has a minimum surface of $0.8$ $\text{m}^2$. Also, for stability reasons, the block weight has to be larger than $3000$ $\text{kg}$. Let’s assume concrete density of $2500$ $\text{kg}/{m}^2$.

## Model

We need to define our model in the form of a nonlinear constrained optimization model to apply `scipy.optimize.minimize`.

We'll define the model as follows:
- Design variables: width, height and depth of a block
- Objective function: minimum volume of the block
- Inequality constraint functions: minimum surface area of each face of $0.8$ $\text{m}^2$ and maximum weight of $3000$ $\text{kg}$
- Equality constraint functions: none
- Bounds: positive dimensions

### Design variables
Let's start with our design variables. In this case a logical choice could be the width, height and depth of our block

$$
x=\left[ \begin{matrix}
   {{x}_{width}}  \\
   {{x}_{depth}}  \\
   {{x}_{height}}  \\
\end{matrix} \right]=\left[ \begin{matrix}
   {{x}_{1}}  \\
   {{x}_{2}}  \\
   {{x}_{3}}  \\
\end{matrix} \right]
$$

### Objective function
Now we can define the objective function as the product of the dimension to represent $\mathop {\min }\limits_x f\left(x\right) $:

$$ \mathop {\min }\limits_x f\left(x\right) = x_1 \cdot x_2 \cdot x_3 $$

### Inequality constraints

Let's continue with the inequality constraints, which should deal with the required positive dimensions, minimum surface area of each face of $0.8$ $\text{m}^2$ and maximum weight of $3000$ $\text{kg}$. These can be defined in the form ${{g}}\left(x_{ij}\right) \le 0$ as:

$$
g_1\left(x\right) = -x_{1} \cdot x_2 + 0.8 \\
g_2\left(x\right) = -x_{2} \cdot x_3 + 0.8 \\
g_3\left(x\right) = -x_{1} \cdot x_3 + 0.8 \\
g_4\left(x\right) = -x_{1} \cdot x_2 \cdot x_3 \cdot 2500 + 3000 \\
$$


<iframe src="https://tudelft.h5p.com/content/1292254701363993127/embed" aria-label="upper lower equal or bigger" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>

### Bounds

The dimensions of the block cannot be negative. Therefore, the bounds can be defined as:

$$
0<{{x}_{i}}\text{   } i=1,2,3
$$

### Find best solution manually

Try and adjust the values for $x_1$, $x_2$ and $x_3$. Can you find the optimal solution?

In [1]:
### Find solution manually

from ipywidgets import widgets, interact
import numpy as np

def func(x):
    vol = x[0]*x[1]*x[2]
    return vol

def nonlinconfun(x):
    c1 = 0.8 - x[0]*x[1]
    c2 = 0.8 - x[0]*x[2]
    c3 = 0.8 - x[1]*x[2]
    c4 = 3000 - 2500 * x[0] * x[1] * x[2]
    return np.array([c1,c2,c3,c4])

def eval(x_1,x_2,x_3):
    x = np.array([x_1,x_2,x_3]) 
    print("Objective function: ", round(func(x),3),"m^3")
    cons = nonlinconfun(x)
    if cons[0] < 0:
        print("Constraint functions 1: ", round(cons[0],2),"m^2","👍")
    else:
        print("Constraint functions 1: ", round(cons[0],2),"m^2","👎")
    if cons[1] < 0:
        print("Constraint functions 2: ", round(cons[1],2),"m^2","👍")
    else:
        print("Constraint functions 2: ", round(cons[1],2),"m^2","👎")
    if cons[2] < 0:
        print("Constraint functions 3: ", round(cons[2],2),"m^2","👍")
    else:
        print("Constraint functions 3: ", round(cons[2],2),"m^2","👎")
    if cons[3] < 0:
        print("Constraint functions 4: ", round(cons[3]),"kg","👍")
    else:
        print("Constraint functions 4: ", round(cons[3]),"kg","👎")
    return

interact(eval,
         
         x_1 = widgets.FloatSlider(min=0, max=5, value=2, step=0.01, description="x_1"),
         x_2 = widgets.FloatSlider(min=0, max=5, value=2, step=0.01, description="x_2"),
         x_3 = widgets.FloatSlider(min=0, max=5, value=2, step=0.01, description="x_3"));

interactive(children=(FloatSlider(value=2.0, description='x_1', max=5.0, step=0.01), FloatSlider(value=2.0, de…

## Method

Now let's solve this problem using an optimization method.

### Import libraries

In [5]:
import scipy as sp 
import numpy as np

### Define variables
As before, we don't need to specify our variable $x$ itself. However, this optimization method requires an initial guess. An arbitrary value is chosen here:

In [6]:
x0 = np.array([5,0,1])

### Define objective function

The objective function gives:

In [7]:
def func(x):
    vol = x[0]*x[1]*x[2]
    return vol

### Define constrain functions

 We had no equality constraints. Unlike before with linear constrained problems, we need an object which defines the upper and lower bounds. As this problem has only an upper bound of $0$, the lower bound is set to $\infty$ which is `np.inf` in python. Note that a single constraint object can include multiple constraints.

In [8]:
def nonlinconfun(x):
    c1 = 0.8 - x[0]*x[1]
    c2 = 0.8 - x[0]*x[2]
    c3 = 0.8 - x[1]*x[2]
    c4 = 3000 - 2500 * x[0] * x[1] * x[2]
    return np.array([c1,c2,c3,c4])

cons = sp.optimize.NonlinearConstraint(nonlinconfun, np.array([-np.inf,-np.inf,-np.inf,-np.inf]), np.array([0,0,0,0]))

### Define bounds
The bounds result in:

In [9]:
bounds = [[0, None],
          [0, None],
          [0, None]]

### Solve the problem

Now let's solve the problem. The `cons` object can be added directly, in the case of equality constraints as well you can define a list of constrainer objects as an input.

In [10]:
result = sp.optimize.minimize(fun = func,x0 = x0,bounds = bounds,constraints=cons)
print(result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.2000000000024624
       x: [ 1.353e+00  1.488e+00  5.962e-01]
     nit: 10
     jac: [ 8.871e-01  8.065e-01  2.013e+00]
    nfev: 41
    njev: 10


## Exercise

<iframe src="https://tudelft.h5p.com/content/1292254715005702697/embed" aria-label="apply" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>